# Importing libraries

In [26]:
import os

import numpy as np
import pandas as pd

import collections
from collections import Counter

# finding the state based on geotags
# from geopy.geocoders import Nominatim

# the Geonamescache library contains information
# about continents, cities and US states
# import geonamescache

import networkx 
from networkx.algorithms.components.connected import connected_components

# Mapping IDs to Images

In [27]:
chicago_data = pd.read_csv("../data/structured/TEXT/Chicago.csv")
chicago_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/Chicago_PostID2Image.csv")

atlanta_data = pd.read_csv("../data/structured/TEXT/Atlanta.csv")
atlanta_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/Atlanta_PostIDToImage.csv")

detroit_data = pd.read_csv("../data/structured/TEXT/Detroit.csv")
detroit_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/Detroit_ID2Post.csv").rename(columns={'Post ID': 'ID', 'Image Title':'IMAGES'})

dallas_data = pd.read_csv("../data/structured/TEXT/Dallas.csv")
dallas_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/Dallas_IDToImage.csv").rename(columns={'Post ID': 'ID', 'Image Title':'IMAGES'})

houston_data = pd.read_csv("../data/structured/TEXT/Houston.csv")
houston_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/Houston_ID2Post.csv")

nyc_data = pd.read_csv("../data/structured/TEXT/New York.csv")
nyc_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/NYC_ID2Image.csv").rename(columns={'Post ID': 'ID', 'Image Title':'IMAGES'})

sf_data = pd.read_csv("../data/structured/TEXT/San Fransisco.csv")
sf_mappings = pd.read_csv("../data/structured/ImgtoCSVmappings/SF_ImageToPost.csv").rename(columns={'Post ID': 'ID', 'Image Title':'IMAGES'})

In [28]:
def map_ids_to_images(data, mappings, save_filename):
    image_dict = {}
    for ids in mappings['ID'].unique():
        df = mappings[mappings['ID']==ids]
        image_dict[ids] = df.IMAGES.to_list()
        
    df = pd.DataFrame({"ID":list(image_dict.keys()), "IMAGES":list(image_dict.values())})
    df['ID'] = df['ID'].astype(float)
    
    df = pd.merge(data[['TEXT', 'PHONES', 'ID']], df, on="ID", how="left")
    df.to_csv(save_filename)

In [29]:
map_ids_to_images(chicago_data, chicago_mappings, "../data/structured/TEXT/Chicago.csv")
map_ids_to_images(atlanta_data, atlanta_mappings, "../data/structured/TEXT/Atlanta.csv")
map_ids_to_images(detroit_data, detroit_mappings, "../data/structured/TEXT/Detroit.csv")
map_ids_to_images(dallas_data, dallas_mappings, "../data/structured/TEXT/Dallas.csv")
map_ids_to_images(houston_data, houston_mappings, "../data/structured/TEXT/Houston.csv")
map_ids_to_images(nyc_data, nyc_mappings, "../data/structured/TEXT/New York.csv")
map_ids_to_images(sf_data, sf_mappings, "../data/structured/TEXT/San Fransisco.csv")

# Loading data

In [31]:
# Loading all the files
filenames = os.listdir("../data/structured/TEXT")

In [38]:
city, samples, phones, images = ([] for i in range(4))
df_dict =  {}

# Getting the stats for advertisements from the cities
for file in filenames:
    if file.endswith('.csv') and file not in ["east.csv", "west.csv", "north.csv", "south.csv", "central.csv"]:
        df = pd.read_csv(os.path.join("../data/structured/TEXT", file))
        city.append(file.replace(".csv", ""))
        samples.append(df.shape[0])
        phone_list = [eval(phone) for phone in df ["PHONES"].to_list()]
        phone_list = [j for i in phone_list for j in i]
        phone_list = [phone for phone in phone_list if len(phone)>=7]
        phones.append(len(phone_list))

        """
        if "IMAGES" in df.columns:
            image_list = [image for image in df.IMAGES.to_list() if str(image) != 'nan']
            print(type(image_list[0]))
            raise Exception("sads")
            if type(image_list[0]) == str:
                image_list = [image.split("|") for image in image_list]
            elif type(image_list[0]) == list:
                image_list = image_list
            else:
                raise Exception("Elements of Image columns inside the dataframe should be either in string or list formats")
            image_list = set([j for i in image_list for j in i])
            images.append(len(image_list))
        else:
            images.append(0)
        """
# df_dict = {"CITY" : city, "ADS": samples, "PHONES": phones, "IMAGES": images}

df_dict = {"CITY" : city,
          "ADS": samples,
          "PHONES": phones,
          "IMAGES": images}

# Getting the stats in Dataframe

In [ ]:
df = pd.DataFrame(df_dict)

In [ ]:
print("Total number of cities:", df.CITY.nunique())
print("Total number of ADS:", df.TEXT.sum())
print("Total number of phones:", df.PHONES.sum())
print("Total number of images:", df.IMAGES.sum())

In [44]:
# dictionary of cities to state mappings
city_to_state_dict = {"East Rancho Dominguez": "California", "Clinton": "Mississippi", "Nanuet": "New York", "Sand Springs": "Oklahoma", "Middle River": "Maryland", "Carbondale": "Illinois", "Boise": "Idaho", "Las Vegas": "Nevada", "Denver": "Colorado", "Hagerstown": "Maryland", "Venice": "Florida", "Moreno Valley": "California", "Mamaroneck": "New York", "Bartow": "Florida", "Bensonhurst": "New York", "Edgewater": "Florida", "Dallas": "Texas", "Benton": "Arkansas", "Lake Havasu City": "Arizona", "New South Memphis": "Tennessee", "North Glendale": "California", "Santee": "California", "Shawnee": "Oklahoma", "North Augusta": "South Carolina", "Brownwood": "Texas", "Methuen": "Massachusetts", "Allapattah": "Florida", "White Oak": "Maryland", "University Place": "Washington", "Muncie": "Indiana", "Naperville": "Illinois", "Hollywood": "Florida", "Richmond West": "Florida", "Bellmore": "New York", "Pine Bluff": "Arkansas", "Natick": "Massachusetts", "Silver Lake": "California", "Ridgecrest": "California", "Parkland": "Washington", "Aberdeen": "Washington", "Cohoes": "New York", "Harlingen": "Texas", "Ennis": "Texas", "North La Crosse": "Wisconsin", "Commerce City": "Colorado", "Racine": "Wisconsin", "Carmel": "Indiana", "Fairview Park": "Ohio", "Queen Creek": "Arizona", "Southglenn": "Colorado", "Wentzville": "Missouri", "Pinecrest": "Florida", "Kingston": "New York", "Syracuse": "New York", "El Cajon": "California", "Champaign": "Illinois", "Stevens Point": "Wisconsin", "San Luis": "Arizona", "Livingston": "New Jersey", "Santa Maria": "California", "Fort Leonard Wood": "Missouri", "West Elkridge": "Maryland", "New Iberia": "Louisiana", "Elk Grove": "California", "Smithtown": "New York", "Greensboro": "North Carolina", "Garden Grove": "California", "Emporia": "Kansas", "Sun Valley": "Nevada", "Wheeling": "West Virginia", "Troy": "Ohio", "Irving": "Connecticut", "Cornelius": "North Carolina", "East Cleveland": "Ohio", "Upland": "California", "Porterville": "California", "Rock Island": "Illinois", "Prattville": "Alabama", "Aloha": "Oregon", "Durham": "North Carolina", "Lynnwood": "Washington", "Dunwoody": "Georgia", "East New York": "New York", "Santa Clara": "California", "Homestead": "Florida", "Borough of Queens": "New York", "Woodland": "California", "Pennsauken": "New Jersey", "Marrero": "Louisiana", "Meadow Woods": "Florida", "Godfrey": "Illinois", "Apache Junction": "Arizona", "Oak Grove": "Oregon", "Surprise": "Arizona", "Lake Oswego": "Oregon", "Goleta": "California", "Tullahoma": "Tennessee", "Kinston": "North Carolina", "Vernon Hills": "Illinois", "Agoura": "California", "Grants Pass": "Oregon", "Topeka": "Kansas", "West New York": "New Jersey", "Grand Forks": "North Dakota", "Cedar Falls": "Iowa", "Minot": "North Dakota", "Carrollton": "Georgia", "West Little River": "Florida", "Huntington Beach": "California", "Newark": "Ohio", "Bellflower": "California", "Wilmington Island": "Georgia", "Country Club": "Florida", "Hazleton": "Pennsylvania", "Burton": "Michigan", "Eureka": "California", "Vincennes": "Indiana", "Monroe": "Washington", "Kernersville": "North Carolina", "Clayton": "Missouri", "Chester": "Pennsylvania", "Caldwell": "Idaho", "Cloverleaf": "Texas", "Brandon": "Mississippi", "Mishawaka": "Indiana", "Winter Springs": "Florida", "Clarksville": "Indiana", "West Warwick": "Rhode Island", "Dunedin": "Florida", "Eldersburg": "Maryland", "Orland Park": "Illinois", "Palm Valley": "Florida", "Holly Springs": "North Carolina", "Maplewood": "New Jersey", "Carrboro": "North Carolina", "Escondido": "California", "Xenia": "Ohio", "Santa Clarita": "California", "Vicksburg": "Mississippi", "Farmers Branch": "Texas", "Mill Creek": "Washington", "Naples": "Florida", "Cedar Hill": "Texas", "Indian Trail": "North Carolina", "Shrewsbury": "Massachusetts", "Mead Valley": "California", "Crown Point": "Indiana", "Blythe": "California", "Centerville": "Utah", "Bothell": "Washington", "American Fork": "Utah", "Casselberry": "Florida", "Chesterfield": "Missouri", "North Olmsted": "Ohio", "Rocky Mount": "North Carolina", "Americus": "Georgia", "Madera": "California", "Port Washington": "New York", "Branford": "Connecticut", "Chickasha": "Oklahoma", "South Bradenton": "Florida", "Oxnard Shores": "California", "Huntley": "Illinois", "Scottsdale": "Arizona", "Palm Coast": "Florida", "Homewood": "Alabama", "Berwyn": "Illinois", "Huber Heights": "Ohio", "Bay Village": "Ohio", "Eagle Pass": "Texas", "Plainview": "Texas", "Gloversville": "New York", "South Portland": "Maine", "Wantagh": "New York", "Staten Island": "New York", "Angleton": "Texas", "Woodburn": "Oregon", "Rockville": "Maryland", "Monterey Park": "California", "Montclair": "New Jersey", "Brockton": "Massachusetts", "Cincinnati": "Ohio", "Somerset": "New Jersey", "Bentonville": "Arkansas", "Elgin": "Illinois", "South Elgin": "Illinois", "Rockland": "Massachusetts", "Bensenville": "Illinois", "San Francisco": "California", "Mukilteo": "Washington", "Merritt Island": "Florida", "Avocado Heights": "California", "Corpus Christi": "Texas", "Altamonte Springs": "Florida", "Ellicott City": "Maryland", "North Royalton": "Ohio", "Toms River": "New Jersey", "Forrest City": "Arkansas", "Kihei": "Hawaii", "Colonia": "New Jersey", "Prairieville": "Louisiana", "South Valley": "New Mexico", "Waipahu": "Hawaii", "Florin": "California", "Rio Linda": "California", "Highlands Ranch": "Colorado", "Crest Hill": "Illinois", "Yarmouth": "Massachusetts", "Augusta": "Georgia", "Bremerton": "Washington", "Lavergne": "Tennessee", "Millcreek": "Utah", "Evanston": "Illinois", "De Pere": "Wisconsin", "Paducah": "Kentucky", "Fremont": "California", "Four Corners": "Oregon", "Lenexa": "Kansas", "Covington": "Washington", "Mustang": "Oklahoma", "Calexico": "California", "Barstow": "California", "Ferndale": "Michigan", "Oklahoma City": "Oklahoma", "Magna": "Utah", "Tahlequah": "Oklahoma", "El Dorado": "Arkansas", "McAlester": "Oklahoma", "Levittown": "New York", "South Whittier": "California", "Huntington": "New York", "McMinnville": "Oregon", "Irondequoit": "New York", "DeBary": "Florida", "Westfield": "Massachusetts", "Castaic": "California", "Villa Park": "Illinois", "Erlanger": "Kentucky", "Bryan": "Texas", "San Jose": "California", "Matthews": "North Carolina", "Maryville": "Tennessee", "Oroville": "California", "Weirton": "West Virginia", "Poinciana": "Florida", "South Peabody": "Massachusetts", "Rohnert Park": "California", "North Kingstown": "Rhode Island", "Spanish Lake": "Missouri", "Hobart": "Indiana", "Rahway": "New Jersey", "Navarre": "Florida", "Harker Heights": "Texas", "Kenosha": "Wisconsin", "West Babylon": "New York", "Marshalltown": "Iowa", "Madison": "New Jersey", "Warrensburg": "Missouri", "Boulder City": "Nevada", "Colleyville": "Texas", "Winchester": "Kentucky", "Lafayette": "Louisiana", "Parsippany": "New Jersey", "Orchards": "Washington", "Center Point": "Alabama", "Crofton": "Maryland", "Royal Palm Beach": "Florida", "Herriman": "Utah", "East Patchogue": "New York", "West Torrington": "Connecticut", "The Crossings": "Florida", "Everett": "Massachusetts", "Austin": "Minnesota", "Tamarac": "Florida", "Arroyo Grande": "California", "Fort Lauderdale": "Florida", "Norfolk": "Nebraska", "Valparaiso": "Indiana", "New Rochelle": "New York", "Vandalia": "Ohio", "Casas Adobes": "Arizona", "Taunton": "Massachusetts", "Morgan Hill": "California", "Council Bluffs": "Iowa", "Cottage Lake": "Washington", "Bethesda": "Maryland", "Centereach": "New York", "Urbana": "Illinois", "Hays": "Kansas", "Lomita": "California", "Ilchester": "Maryland", "Webster Groves": "Missouri", "Southfield": "Michigan", "Lawton": "Oklahoma", "Carol City": "Florida", "North Bethesda": "Maryland", "Lake Magdalene": "Florida", "Rio Rico": "Arizona", "Avon Center": "Ohio", "Skokie": "Illinois", "Logan": "Utah", "Amarillo": "Texas", "Shoreline": "Washington", "Coppell": "Texas", "Pearl River": "New York", "Titusville": "Florida", "Orem": "Utah", "Cary": "Illinois", "Fairview Heights": "Illinois", "Acworth": "Georgia", "Golden": "Colorado", "Gardner": "Kansas", "Lombard": "Illinois", "Dania Beach": "Florida", "Maple Heights": "Ohio", "Casa de Oro-Mount Helix": "California", "Brownsville": "Florida", "Chicago": "Illinois", "Largo": "Florida", "Papillion": "Nebraska", "Menomonee Falls": "Wisconsin", "Jacksonville Beach": "Florida", "Hopkinsville": "Kentucky", "Ocala": "Florida", "Defiance": "Ohio", "Texarkana": "Arkansas", "Warwick": "Rhode Island", "Bonita Springs": "Florida", "Garner": "North Carolina", "Vineyard": "California", "Great Falls": "Montana", "Downey": "California", "West Haven": "Connecticut", "Goose Creek": "South Carolina", "Menasha": "Wisconsin", "Fountain Valley": "California", "Evansville": "Indiana", "Radcliff": "Kentucky", "Tampa": "Florida", "Sherman Oaks": "California", "Aliso Viejo": "California", "Belvidere": "Illinois", "Lithia Springs": "Georgia", "Russellville": "Arkansas", "Swansea": "Massachusetts", "Ferguson": "Missouri", "Tigard": "Oregon", "Bradenton": "Florida", "Geneva": "Illinois", "Salt Lake City": "Utah", "Grove City": "Ohio", "Calverton": "Maryland", "Eastpointe": "Michigan", "Fallbrook": "California", "Payson": "Arizona", "Melrose": "Massachusetts", "Palm Desert": "California", "State College": "Pennsylvania", "Marion": "Ohio", "Rochester": "New York", "Milledgeville": "Georgia", "New Smyrna Beach": "Florida", "Severn": "Maryland", "Columbus": "Georgia", "Junction City": "Kansas", "West Valley City": "Utah", "Sudbury": "Massachusetts", "Hauppauge": "New York", "Greeley": "Colorado", "Altus": "Oklahoma", "Pompano Beach": "Florida", "Mountain View": "California", "Manhattan": "New York", "South Lake Tahoe": "California", "Wausau": "Wisconsin", "Ridgewood": "New Jersey", "Hacienda Heights": "California", "Miami Gardens": "Florida", "Belton": "Missouri", "Wooster": "Ohio", "Sunnyvale": "California", "Channelview": "Texas", "Waukesha": "Wisconsin", "Fishers": "Indiana", "Brighton": "Colorado", "Calhoun": "Georgia", "Fort Hood": "Texas", "Owings Mills": "Maryland", "Lauderdale Lakes": "Florida", "Roosevelt": "New York", "East Independence": "Missouri", "Juneau": "Alaska", "Golden Gate": "Florida", "Foster City": "California", "Kalispell": "Montana", "South Hadley": "Massachusetts", "Findlay": "Ohio", "Huntington Station": "New York", "Trussville": "Alabama", "Baldwin Park": "California", "Prairie Village": "Kansas", "LaPorte": "Indiana", "Newport": "Rhode Island", "Lebanon": "Tennessee", "South Holland": "Illinois", "Albuquerque": "New Mexico", "Hattiesburg": "Mississippi", "Chowchilla": "California", "Sun City West": "Arizona", "Lincoln Park": "Michigan", "Carlsbad": "New Mexico", "Tallahassee": "Florida", "Buenaventura Lakes": "Florida", "Ceres": "California", "Mercer Island": "Washington", "Talladega": "Alabama", "Fairland": "Maryland", "San Dimas": "California", "Rapid City": "South Dakota", "Newburg": "Kentucky", "Brent": "Florida", "Yuma": "Arizona", "Northridge": "California", "Paragould": "Arkansas", "West Gulfport": "Mississippi", "Euless": "Texas", "Lake Stevens": "Washington", "Goodings Grove": "Illinois", "Bel Air South": "Maryland", "Haltom City": "Texas", "Camp Springs": "Maryland", "Troutdale": "Oregon", "National City": "California", "Edinburg": "Texas", "Point Pleasant": "New Jersey", "La Grange": "Illinois", "Socorro Mission Number 1 Colonia": "Texas", "Cape Girardeau": "Missouri", "Upper Saint Clair": "Pennsylvania", "Florence": "Alabama", "Bridgewater": "New Jersey", "Raymore": "Missouri", "New Berlin": "Wisconsin", "Dearborn Heights": "Michigan", "Detroit": "Michigan", "Flint": "Michigan", "Tulare": "California", "Marlboro": "New Jersey", "Castro Valley": "California", "Lino Lakes": "Minnesota", "Calumet City": "Illinois", "Peekskill": "New York", "Mequon": "Wisconsin", "Roseville": "California", "Fort Hunt": "Virginia", "Reisterstown": "Maryland", "Moorhead": "Minnesota", "Mooresville": "North Carolina", "Natchitoches": "Louisiana", "Agawam": "Massachusetts", "Crestview": "Florida", "Willingboro": "New Jersey", "Hoffman Estates": "Illinois", "Sterling Heights": "Michigan", "Cliffside Park": "New Jersey", "Nashua": "New Hampshire", "Madison Heights": "Michigan", "New York City": "New York", "Palmetto Bay": "Florida", "Dundalk": "Maryland", "Calabasas": "California", "Olney": "Maryland", "Maryland Heights": "Missouri", "Lewiston Orchards": "Idaho", "Newnan": "Georgia", "Glen Burnie": "Maryland", "Sun Prairie": "Wisconsin", "Cockeysville": "Maryland", "North Creek": "Washington", "Derby": "Kansas", "Coconut Creek": "Florida", "Murfreesboro": "Tennessee", "Parker": "Colorado", "Arbutus": "Maryland", "Loveland": "Colorado", "Susanville": "California", "Chalmette": "Louisiana", "Saint Cloud": "Florida", "Edison": "New Jersey", "Cedar Rapids": "Iowa", "Beaumont": "Texas", "Flowing Wells": "Arizona", "Patterson": "California", "Summit": "New Jersey", "Pinewood": "Florida", "Owosso": "Michigan", "Antelope": "California", "Greer": "South Carolina", "Jackson": "Michigan", "Dodge City": "Kansas", "Ponte Vedra Beach": "Florida", "Palatine": "Illinois", "La Crosse": "Wisconsin", "Columbia Heights": "Minnesota", "Pinellas Park": "Florida", "Chico": "California", "Kuna": "Idaho", "Gretna": "Louisiana", "Ewing": "New Jersey", "Parole": "Maryland", "Hannibal": "Missouri", "Hillsboro": "Oregon", "Alamogordo": "New Mexico", "Carol Stream": "Illinois", "Willoughby": "Ohio", "South Bend": "Indiana", "Pacific Grove": "California", "Jamaica Plain": "Massachusetts", "Griffith": "Indiana", "North Bellmore": "New York", "South Euclid": "Ohio", "Valley Stream": "New York", "O'Fallon": "Illinois", "Onalaska": "Wisconsin", "Crawfordsville": "Indiana", "Aldine": "Texas", "Post Falls": "Idaho", "Rexburg": "Idaho", "Ponca City": "Oklahoma", "Kaysville": "Utah", "St. Marys": "Georgia", "Shenandoah": "Louisiana", "Westbury": "New York", "Wasco": "California", "Maitland": "Florida", "Fortuna Foothills": "Arizona", "Frankfort": "Kentucky", "Battle Creek": "Michigan", "Paterson": "New Jersey", "Tualatin": "Oregon", "Murrysville": "Pennsylvania", "Balch Springs": "Texas", "Rio Rancho": "New Mexico", "Selden": "New York", "Jersey City": "New Jersey", "Chandler": "Arizona", "Fort Walton Beach": "Florida", "New Milford": "New Jersey", "Newberg": "Oregon", "Affton": "Missouri", "Kaneohe": "Hawaii", "Hinesville": "Georgia", "Lexington-Fayette": "Kentucky", "Nutley": "New Jersey", "Pekin": "Illinois", "Summerville": "South Carolina", "Lake Forest": "Illinois", "Glendale": "California", "Flower Mound": "Texas", "Fond du Lac": "Wisconsin", "Ruston": "Louisiana", "Valrico": "Florida", "Melrose Park": "Illinois", "Radnor": "Pennsylvania", "Shelbyville": "Indiana", "Henderson": "North Carolina", "Raleigh": "North Carolina", "Schaumburg": "Illinois", "Perry Hall": "Maryland", "West Bloomfield Township": "Michigan", "Terrytown": "Louisiana", "Rancho Cordova": "California", "Bella Vista": "Arkansas", "Olive Branch": "Mississippi", "Cocoa": "Florida", "Longview": "Washington", "Hamtramck": "Michigan", "Randallstown": "Maryland", "Hesperia": "California", "Five Corners": "Washington", "East Brainerd": "Tennessee", "Mineola": "New York", "Opa-locka": "Florida", "Littleton": "Colorado", "South Jordan Heights": "Utah", "Albemarle": "North Carolina", "Gillette": "Wyoming", "North Platte": "Nebraska", "Montgomery": "Alabama", "North Highlands": "California", "Brea": "California", "South Miami Heights": "Florida", "Avenal": "California", "Lancaster": "Ohio", "Brownsburg": "Indiana", "Ormond Beach": "Florida", "Coos Bay": "Oregon", "Pembroke Pines": "Florida", "Claremore": "Oklahoma", "Gastonia": "North Carolina", "East Saint Louis": "Illinois", "Sterling": "Illinois", "Lake Worth Corridor": "Florida", "Selma": "California", "Seven Oaks": "South Carolina", "Rancho Mirage": "California", "Wake Forest": "North Carolina", "Gahanna": "Ohio", "Richmond": "Kentucky", "Park Forest": "Illinois", "Wethersfield": "Connecticut", "Douglasville": "Georgia", "Laurel": "Mississippi", "Homer Glen": "Illinois", "Fountain": "Colorado", "San Marcos": "California", "Copiague": "New York", "Matteson": "Illinois", "North Tonawanda": "New York", "Stockton": "California", "Statesboro": "Georgia", "Fargo": "North Dakota", "West Lake Stevens": "Washington", "Corona": "California", "Cutler Bay": "Florida", "Seaside": "California", "Shively": "Kentucky", "Hemet": "California", "Billerica": "Massachusetts", "Amesbury": "Massachusetts", "Spring Valley": "New York", "Chino": "California", "Great Bend": "Kansas", "Bossier City": "Louisiana", "Windsor": "Colorado", "Riverside": "Ohio", "Limerick": "Pennsylvania", "Blaine": "Minnesota", "Paramount": "California", "Lynbrook": "New York", "Eggertsville": "New York", "Ballwin": "Missouri", "Gresham": "Oregon", "Saint Charles": "Missouri", "Saint Matthews": "Kentucky", "Nashville": "Tennessee", "North Babylon": "New York", "Holtsville": "New York", "Southchase": "Florida", "Sanger": "California", "Eloy": "Arizona", "Daly City": "California", "Bedford": "New Hampshire", "Kennewick": "Washington", "Cranston": "Rhode Island", "Garden City": "New York", "Wellesley": "Massachusetts", "Ypsilanti": "Michigan", "Tinton Falls": "New Jersey", "Alamo": "Texas", "Bryn Mawr-Skyway": "Washington", "Roswell": "Georgia", "Blue Island": "Illinois", "West Bend": "Wisconsin", "Alvin": "Texas", "Culver City": "California", "Hoboken": "New Jersey", "Atascocita": "Texas", "Ridgeland": "Mississippi", "Wheaton": "Maryland", "East Millcreek": "Utah", "Ashwaubenon": "Wisconsin", "West Lafayette": "Indiana", "Worcester": "Massachusetts", "Rocky River": "Ohio", "Keystone": "Florida", "Fayetteville": "Georgia", "Laconia": "New Hampshire", "Rosemont": "California", "Spanish Springs": "Nevada", "Fruit Cove": "Florida", "Opelousas": "Louisiana", "Jeffersontown": "Kentucky", "Banning": "California", "Lacey": "Washington", "Oakdale": "Minnesota", "Tucson": "Arizona", "McDonough": "Georgia", "Mahwah": "New Jersey", "Providence": "Rhode Island", "Rochester Hills": "Michigan", "North Salt Lake": "Utah", "Liberty": "Missouri", "San Lorenzo": "California", "Ludlow": "Massachusetts", "Macon": "Georgia", "Georgetown": "Texas", "Schererville": "Indiana", "Sikeston": "Missouri", "Jamestown": "New York", "Eau Claire": "Wisconsin", "Lauderhill": "Florida", "Metairie Terrace": "Louisiana", "Fitchburg": "Wisconsin", "Acton": "Massachusetts", "Westerly": "Rhode Island", "West Chester": "Pennsylvania", "Denison": "Texas", "Diamond Bar": "California", "Goldsboro": "North Carolina", "Chambersburg": "Pennsylvania", "Merrick": "New York", "Dolton": "Illinois", "Hendersonville": "Tennessee", "Vero Beach South": "Florida", "Redford": "Michigan", "Broomfield": "Colorado", "Canoga Park": "California", "Del City": "Oklahoma", "Shorewood": "Illinois", "Kingsport": "Tennessee", "Akron": "Ohio", "Ensley": "Florida", "Mount Vernon": "New York", "Piscataway": "New Jersey", "Ellensburg": "Washington", "Cahokia": "Illinois", "Kissimmee": "Florida", "Dothan": "Alabama", "San Pablo": "California", "Kahului": "Hawaii", "Del Rio": "Texas", "Alice": "Texas", "Menlo Park": "California", "South Salt Lake": "Utah", "Broadview Heights": "Ohio", "Chanhassen": "Minnesota", "Haverhill": "Massachusetts", "Tulsa": "Oklahoma", "High Point": "North Carolina", "San Rafael": "California", "Clovis": "California", "Ottawa": "Illinois", "Des Plaines": "Illinois", "Redland": "Maryland", "West Orange": "New Jersey", "South Vineland": "New Jersey", "Baton Rouge": "Louisiana", "Butte": "Montana", "East Longmeadow": "Massachusetts", "Los Banos": "California", "Milwaukie": "Oregon", "Santa Fe": "New Mexico", "Chattanooga": "Tennessee", "Spokane Valley": "Washington", "Coachella": "California", "Mission Viejo": "California", "Severna Park": "Maryland", "Algonquin": "Illinois", "Green": "Ohio", "Mehlville": "Missouri", "Rancho Palos Verdes": "California", "Carpentersville": "Illinois", "Aurora": "Ohio", "Dedham": "Massachusetts", "Palmer": "Massachusetts", "Eastlake": "Ohio", "Syosset": "New York", "Dixiana": "Alabama", "East Setauket": "New York", "Easthampton": "Massachusetts", "Livonia": "Michigan", "Silverdale": "Washington", "Catonsville": "Maryland", "Fountainebleau": "Florida", "New Hope": "Minnesota", "Seminole": "Florida", "Town 'n' Country": "Florida", "Corinth": "Texas", "Lindenhurst": "New York", "Liberal": "Kansas", "Framingham Center": "Massachusetts", "Pearl City": "Hawaii", "Casa Grande": "Arizona", "Mililani Town": "Hawaii", "Ojus": "Florida", "Kendale Lakes": "Florida", "Fairbanks": "Alaska", "Essex": "Maryland", "Three Lakes": "Florida", "Beverly Cove": "Massachusetts", "Taylors": "South Carolina", "Vestavia Hills": "Alabama", "Salmon Creek": "Washington", "Dix Hills": "New York", "Alameda": "California", "Mountain Brook": "Alabama", "Sicklerville": "New Jersey", "Groves": "Texas", "East Chicago": "Indiana", "Strongsville": "Ohio", "Rotterdam": "New York", "Plantation": "Florida", "Bay City": "Michigan", "West Raleigh": "North Carolina", "Easton": "Massachusetts", "Union City": "New Jersey", "Nampa": "Idaho", "Bartlesville": "Oklahoma", "Norristown": "Pennsylvania", "Jonesboro": "Arkansas", "Waterloo": "Iowa", "Lehi": "Utah", "Holiday": "Florida", "Mount Juliet": "Tennessee", "Estelle": "Louisiana", "Woodbury": "Minnesota", "Belmont": "Massachusetts", "Menifee": "California", "Munster": "Indiana", "Tracy": "California", "Sheboygan": "Wisconsin", "Wellington": "Florida", "Bloomfield": "New Jersey", "Addison": "Illinois", "Tiffin": "Ohio", "City of Sammamish": "Washington", "Taylor": "Michigan", "Bristol": "Rhode Island", "Jasper": "Indiana", "Saratoga Springs": "New York", "Egypt Lake-Leto": "Florida", "Fall River": "Massachusetts", "Garland": "Texas", "Sandy Hills": "Utah", "Elkton": "Maryland", "Harrison": "New York", "Big Spring": "Texas", "Coronado": "California", "Rockville Centre": "New York", "Roseburg": "Oregon", "Perrysburg": "Ohio", "Miami Beach": "Florida", "Scranton": "Pennsylvania", "Centralia": "Washington", "Muskogee": "Oklahoma", "Edina": "Minnesota", "North Aurora": "Illinois", "Smyrna": "Georgia", "Omaha": "Nebraska", "Avon": "Ohio", "Clarksdale": "Mississippi", "Sandusky": "Ohio", "Hazel Park": "Michigan", "Tempe": "Arizona", "West Covina": "California", "Grandville": "Michigan", "Pahrump": "Nevada", "Temple Terrace": "Florida", "Norton": "Massachusetts", "Dinuba": "California", "Kirksville": "Missouri", "Fuquay-Varina": "North Carolina", "Oxnard": "California", "Apopka": "Florida", "New Albany": "Indiana", "Old Bridge": "New Jersey", "Stoneham": "Massachusetts", "Jenison": "Michigan", "West Hills": "California", "Annapolis": "Maryland", "Jefferson City": "Missouri", "Southaven": "Mississippi", "Chaska": "Minnesota", "Appleton": "Wisconsin", "Ansonia": "Connecticut", "Hopatcong Hills": "New Jersey", "Cimarron Hills": "Colorado", "Orinda": "California", "New Castle": "Pennsylvania", "Altamont": "Oregon", "Noblesville": "Indiana", "Edmonds": "Washington", "Irvington": "New Jersey", "Mount Clemens": "Michigan", "Knoxville": "Tennessee", "Longmeadow": "Massachusetts", "Harrisburg": "Pennsylvania", "Hutchinson": "Kansas", "Saint Clair Shores": "Michigan", "San Carlos": "California", "Tustin": "California", "Claremont": "California", "Immokalee": "Florida", "West Des Moines": "Iowa", "Lehigh Acres": "Florida", "East Hill-Meridian": "Washington", "Gadsden": "Alabama", "Orcutt": "California", "Plainfield": "Illinois", "San Leandro": "California", "New Philadelphia": "Ohio", "Broken Arrow": "Oklahoma", "Ada": "Oklahoma", "Oakland Park": "Florida", "Palmdale": "California", "Fairfield Heights": "Indiana", "Central Islip": "New York", "Cathedral City": "California", "Sunset": "Florida", "Nogales": "Arizona", "Jenks": "Oklahoma", "Oildale": "California", "Kent": "Washington", "Fridley": "Minnesota", "Cudahy": "Wisconsin", "Whitehall Township": "Pennsylvania", "Lawrence": "Massachusetts", "Oshkosh": "Wisconsin", "San Bruno": "California", "Dalton": "Georgia", "Lake in the Hills": "Illinois", "Honolulu": "Hawaii", "Iselin": "New Jersey", "Sandy Springs": "Georgia", "Mastic": "New York", "Frederickson": "Washington", "Westland": "Michigan", "Oxon Hill": "Maryland", "Asbury Park": "New Jersey", "Cranberry Township": "Pennsylvania", "Antioch": "California", "Central Point": "Oregon", "South Burlington": "Vermont", "Sammamish": "Washington", "Catalina Foothills": "Arizona", "Oak Lawn": "Illinois", "Central": "Louisiana", "Damascus": "Maryland", "Kokomo": "Indiana", "Santa Ana": "California", "Louisville": "Colorado", "Clermont": "Florida", "Framingham": "Massachusetts", "East Chattanooga": "Tennessee", "Olathe": "Kansas", "South Bel Air": "Maryland", "Franklin Park": "Illinois", "Newburgh": "New York", "South Yuba City": "California", "Morton Grove": "Illinois", "Socastee": "South Carolina", "Fort Bragg": "North Carolina", "Kendall": "Florida", "Hopatcong": "New Jersey", "Valley Station": "Kentucky", "Beavercreek": "Ohio", "South Hill": "Washington", "Massapequa Park": "New York", "El Segundo": "California", "Pasco": "Washington", "Norwood": "Ohio", "Brookings": "South Dakota", "Fernley": "Nevada", "West Jordan": "Utah", "Novi": "Michigan", "Caledonia": "Wisconsin", "North Bay Shore": "New York", "Twinsburg": "Ohio", "Biloxi": "Mississippi", "Albert Lea": "Minnesota", "Machesney Park": "Illinois", "Clifton": "New Jersey", "North Lauderdale": "Florida", "Merrimack": "New Hampshire", "Kearny": "New Jersey", "Arcata": "California", "East Lake": "Florida", "Hope Mills": "North Carolina", "Woodbridge": "New Jersey", "Suitland": "Maryland", "Janesville": "Wisconsin", "Phoenix": "Arizona", "West Fargo": "North Dakota", "Lumberton": "North Carolina", "Libertyville": "Illinois", "Grand Island": "Nebraska", "McKeesport": "Pennsylvania", "Elko": "Nevada", "Portage": "Indiana", "Folsom": "California", "Langley Park": "Maryland", "Massillon": "Ohio", "Waverly": "Michigan", "Eagle": "Idaho", "Warren Township": "New Jersey", "Meridian": "Idaho", "Chelmsford": "Massachusetts", "Yakima": "Washington", "Longmont": "Colorado", "White Plains": "New York", "Saginaw": "Michigan", "Silver Spring": "Maryland", "North Bergen": "New Jersey", "Cedar Park": "Texas", "SeaTac": "Washington", "Placentia": "California", "Davis": "California", "Douglas": "Arizona", "Snellville": "Georgia", "Farmington Hills": "Michigan", "Evans": "Georgia", "West Milford": "New Jersey", "St. Louis": "Missouri", "Prunedale": "California", "Milwaukee": "Wisconsin", "Westlake": "Ohio", "Depew": "New York", "Friendswood": "Texas", "Johnstown": "Pennsylvania", "Mount Pleasant": "Michigan", "Las Cruces": "New Mexico", "Lakeville": "Minnesota", "Manhattan Beach": "California", "Opportunity": "Washington", "Gurnee": "Illinois", "North Charleston": "South Carolina", "Soledad": "California", "Thomasville": "North Carolina", "Riverview": "Florida", "Canon City": "Colorado", "Truckee": "California", "Lockport": "New York", "Columbia": "Maryland", "Rockledge": "Florida", "Wildwood": "Missouri", "Glen Cove": "New York", "Lynn": "Massachusetts", "Wisconsin Rapids": "Wisconsin", "Owatonna": "Minnesota", "Arcadia": "California", "San Ramon": "California", "North Providence": "Rhode Island", "Owasso": "Oklahoma", "Santa Cruz": "California", "Goodlettsville": "Tennessee", "Moore": "Oklahoma", "Winter Haven": "Florida", "Red Wing": "Minnesota", "Northbrook": "Illinois", "Hermiston": "Oregon", "Reedley": "California", "Candler-McAfee": "Georgia", "South San Francisco": "California", "Fort Wayne": "Indiana", "Simi Valley": "California", "Bridgeton": "New Jersey", "Iowa City": "Iowa", "Lochearn": "Maryland", "Atlantic City": "New Jersey", "Beacon": "New York", "Camas": "Washington", "Mankato": "Minnesota", "Martha Lake": "Washington", "Lakeside": "Florida", "Glenview": "Illinois", "Greater Northdale": "Florida", "Klamath Falls": "Oregon", "Golden Glades": "Florida", "Hazel Dell": "Washington", "Okemos": "Michigan", "Galesburg": "Illinois", "Greeneville": "Tennessee", "Horn Lake": "Mississippi", "Auburn": "Massachusetts", "Kentwood": "Michigan", "Pampa": "Texas", "Gainesville": "Texas", "King of Prussia": "Pennsylvania", "East Riverdale": "Maryland", "Edwardsville": "Illinois", "Anchorage": "Alaska", "Oak Hill": "Virginia", "Fort Myers": "Florida", "Moses Lake": "Washington", "Grafton": "Massachusetts", "Westminster": "Maryland", "Springfield": "Massachusetts", "Pickerington": "Ohio", "New Bedford": "Massachusetts", "Moorpark": "California", "City of Milford (balance)": "Connecticut", "Evergreen Park": "Illinois", "Dayton": "Ohio", "Pleasure Ridge Park": "Kentucky", "Holbrook": "New York", "Holland": "Michigan", "Sidney": "Ohio", "West Odessa": "Texas", "Secaucus": "New Jersey", "Jupiter": "Florida", "Johnson City": "Tennessee", "Gulfport": "Mississippi", "Ardmore": "Oklahoma", "Minnetonka Mills": "Minnesota", "Myrtle Beach": "South Carolina", "Sahuarita": "Arizona", "Palm Springs": "California", "Pelham": "Alabama", "Ontario": "California", "Ham Lake": "Minnesota", "Birmingham": "Alabama", "Upper Arlington": "Ohio", "North Ridgeville": "Ohio", "Marietta": "Georgia", "Wakefield": "Massachusetts", "Imperial Beach": "California", "Huntsville": "Alabama", "Pico Rivera": "California", "Pearl": "Mississippi", "Cameron Park": "California", "Ankeny": "Iowa", "Lompoc": "California", "Okolona": "Kentucky", "Maywood": "Illinois", "Loves Park": "Illinois", "Martinez": "California", "Norland": "Florida", "Cranford": "New Jersey", "Lansdale": "Pennsylvania", "Shoreview": "Minnesota", "Independence": "Kentucky", "Orange": "New Jersey", "Hudson": "Ohio", "Stuart": "Florida", "Woodridge": "Illinois", "Suisun": "California", "Cherry Hill": "New Jersey", "Inver Grove Heights": "Minnesota", "Highland Park": "Illinois", "Gloucester": "Massachusetts", "Elkridge": "Maryland", "Brooklyn": "New York", "Teaneck": "New Jersey", "Wolcott": "Connecticut", "Vero Beach": "Florida", "Lynwood": "California", "Redding": "California", "East Pensacola Heights": "Florida", "Cheektowaga": "New York", "Oakville": "Missouri", "Broad Ripple": "Indiana", "Grand Junction": "Colorado", "Oswego": "New York", "Terre Haute": "Indiana", "Glen Avon": "California", "Bullhead City": "Arizona", "Monsey": "New York", "Delray Beach": "Florida", "Manitowoc": "Wisconsin", "Drexel Heights": "Arizona", "Frederick": "Maryland", "Rock Springs": "Wyoming", "Pottstown": "Pennsylvania", "Key West": "Florida", "Sayville": "New York", "Leominster": "Massachusetts", "Riviera Beach": "Florida", "Howard": "Wisconsin", "Highland": "Indiana", "Englewood": "New Jersey", "Socorro": "Texas", "Bayonne": "New Jersey", "West Springfield": "Massachusetts", "Bear": "Delaware", "Bessemer": "Alabama", "Holt": "Michigan", "Issaquah": "Washington", "Newburyport": "Massachusetts", "Bolingbrook": "Illinois", "Raytown": "Missouri", "New Brighton": "Minnesota", "West Whittier-Los Nietos": "California", "Murray": "Kentucky", "Elmont": "New York", "Odessa": "Texas", "Burleson": "Texas", "Paradise": "California", "Petaluma": "California", "Brawley": "California", "Maple Valley": "Washington", "Brooklyn Center": "Minnesota", "Delaware": "Ohio", "Zanesville": "Ohio", "Prior Lake": "Minnesota", "Union": "New Jersey", "Hastings": "Nebraska", "Fountain Hills": "Arizona", "Greenfield": "Wisconsin", "Rosemead": "California", "Altoona": "Pennsylvania", "Millbrae": "California", "Bangor": "Maine", "Shelby": "Michigan", "Pleasant Prairie": "Wisconsin", "Spring Hill": "Florida", "Central Falls": "Rhode Island", "Perth Amboy": "New Jersey", "Hackensack": "New Jersey", "Boynton Beach": "Florida", "North Hollywood": "California", "Greenacres City": "Florida", "Garfield": "New Jersey", "Hilton Head Island": "South Carolina", "South Portland Gardens": "Maine", "Bel Air North": "Maryland", "Morrisville": "North Carolina", "Silver Firs": "Washington", "Bonney Lake": "Washington", "Brentwood": "California", "Saint Paul": "Minnesota", "Coeur d'Alene": "Idaho", "Potomac": "Maryland", "Hollister": "California", "Pleasant Hill": "California", "Land O' Lakes": "Florida", "Palm Beach Gardens": "Florida", "Brookhaven": "Georgia", "Hallandale Beach": "Florida", "Champlin": "Minnesota", "East Hemet": "California", "Piqua": "Ohio", "Carrollwood": "Florida", "Helena": "Montana", "Mint Hill": "North Carolina", "San Angelo": "Texas", "Montrose": "Colorado", "Mountain Top": "Pennsylvania", "New Bern": "North Carolina", "Dickinson": "North Dakota", "West and East Lealman": "Florida", "Haslett": "Michigan", "Dublin": "California", "South Ogden": "Utah", "Rancho Cucamonga": "California", "Castle Rock": "Colorado", "Winter Garden": "Florida", "Steubenville": "Ohio", "Greenbelt": "Maryland", "Chicopee": "Massachusetts", "Monrovia": "California", "Hanford": "California", "Plymouth": "Minnesota", "Shafter": "California", "Iron River": "Michigan", "Eustis": "Florida", "Hillside": "New Jersey", "Urbandale": "Iowa", "Wright": "Florida", "Scarsdale": "New York", "Green Haven": "Maryland", "Anthem": "Arizona", "Tempe Junction": "Arizona", "San Fernando": "California", "Davie": "Florida", "Punta Gorda": "Florida", "Ferry Pass": "Florida", "Lincoln": "Nebraska", "Lakeland": "Florida", "Park Ridge": "Illinois", "Midwest City": "Oklahoma", "Shirley": "New York", "Allentown": "Pennsylvania", "Collierville": "Tennessee", "El Mirage": "Arizona", "Muskegon": "Michigan", "El Paso": "Texas", "Carson City": "Nevada", "Muscatine": "Iowa", "University City": "Missouri", "Moraga": "California", "Painesville": "Ohio", "East Meadow": "New York", "Cupertino": "California", "American Canyon": "California", "Wyandotte": "Michigan", "Pullman": "Washington", "Fort Dodge": "Iowa", "Prichard": "Alabama", "Hazelwood": "Missouri", "North Little Rock": "Arkansas", "Lake Butler": "Florida", "Tinley Park": "Illinois", "Gallatin": "Tennessee", "Sherwood": "Oregon", "Lodi": "New Jersey", "Kankakee": "Illinois", "Drexel Hill": "Pennsylvania", "Sartell": "Minnesota", "Lewiston": "Maine", "Conway": "Arkansas", "Tifton": "Georgia", "Tacoma": "Washington", "Milton": "Massachusetts", "Saint Michael": "Minnesota", "Philadelphia": "Pennsylvania", "North Amityville": "New York", "Copperas Cove": "Texas", "North Peoria": "Illinois", "Little Rock": "Arkansas", "Poway": "California", "Brushy Creek": "Texas", "Wheat Ridge": "Colorado", "Cantonment": "Florida", "New Brunswick": "New Jersey", "Chillum": "Maryland", "Mundelein": "Illinois", "Nicholasville": "Kentucky", "Kiryas Joel": "New York", "Huntington Park": "California", "Maple Shade": "New Jersey", "Charlotte": "North Carolina", "Phoenixville": "Pennsylvania", "Sunnyside": "Washington", "Boulder": "Colorado", "Ashtabula": "Ohio", "Glassboro": "New Jersey", "Danvers": "Massachusetts", "Mason City": "Iowa", "Abilene": "Texas", "Anoka": "Minnesota", "Lisle": "Illinois", "Cabot": "Arkansas", "Walnut Park": "California", "Bayou Cane": "Louisiana", "Winthrop": "Massachusetts", "Salinas": "California", "Manchester": "New Hampshire", "Lawrenceville": "Georgia", "Southbridge": "Massachusetts", "Alafaya": "Florida", "Brigham City": "Utah", "Redwood City": "California", "Plattsburgh": "New York", "Olympia": "Washington", "Glendora": "California", "Willmar": "Minnesota", "Walla Walla": "Washington", "Saugus": "Massachusetts", "Parma Heights": "Ohio", "Searcy": "Arkansas", "Long Beach": "California", "Whittier": "California", "Buffalo Grove": "Illinois", "Highview": "Kentucky", "San Pedro": "California", "Bloomington": "Indiana", "Santa Rosa": "California", "Adelphi": "Maryland", "Pinole": "California", "Pasadena": "California", "Floral Park": "New York", "Roselle": "Illinois", "Wilsonville": "Oregon", "Carmichael": "California", "Burlington": "Kentucky", "Salisbury": "Maryland", "Van Buren": "Arkansas", "Attleboro": "Massachusetts", "Hinsdale": "Illinois", "Stillwater": "Oklahoma", "Maryland City": "Maryland", "Milford": "Massachusetts", "Greenburgh": "New York", "Kaukauna": "Wisconsin", "Scottsbluff": "Nebraska", "Midvale": "Utah", "Canby": "Oregon", "Binghamton": "New York", "Burien": "Washington", "Florence-Graham": "California", "Mentor": "Ohio", "Lakewood": "Washington", "Oak Park": "Michigan", "Fullerton": "California", "Citrus Heights": "California", "Westerville": "Ohio", "Seymour": "Indiana", "Pueblo": "Colorado", "Fair Oaks": "California", "Milford Mill": "Maryland", "Corvallis": "Oregon", "Bartlett": "Illinois", "Deerfield Beach": "Florida", "East Lansing": "Michigan", "Sycamore": "Illinois", "Colchester": "Vermont", "Middletown": "Rhode Island", "Washington, D.C.": "District of Columbia", "South Gate": "California", "Medford": "Oregon", "Oak Harbor": "Washington", "Jacksonville": "Arkansas", "Alpharetta": "Georgia", "Artesia": "California", "Franklin": "Massachusetts", "Rubidoux": "California", "Reading": "Massachusetts", "Wyoming": "Michigan", "Marblehead": "Massachusetts", "South Laurel": "Maryland", "Fords": "New Jersey", "Woodland Hills": "California", "Amsterdam": "New York", "Saint George": "Utah", "Pace": "Florida", "Lemay": "Missouri", "Roy": "Utah", "Bellaire": "Texas", "Gallup": "New Mexico", "San Mateo": "California", "Hanover Park": "Illinois", "Bowling Green": "Ohio", "East Palo Alto": "California", "Puyallup": "Washington", "Eden": "North Carolina", "Orangevale": "California", "Coon Rapids": "Minnesota", "Baltimore": "Maryland", "South Pasadena": "California", "West Palm Beach": "Florida", "San Juan Capistrano": "California", "Pleasantville": "New Jersey", "Pleasanton": "California", "Wildomar": "California", "Whitney": "Nevada", "Ozark": "Missouri", "Iona": "Florida", "Boston": "Massachusetts", "Salem": "Oregon", "Lemont": "Illinois", "Carteret": "New Jersey", "Bellevue": "Nebraska", "Waldorf": "Maryland", "Grand Rapids": "Michigan", "Golden Valley": "Minnesota", "Streetsboro": "Ohio", "Schenectady": "New York", "Kettering": "Ohio", "Huntersville": "North Carolina", "Arlington": "Washington", "Beverly Hills": "California", "Beloit": "Wisconsin", "Arvada": "Colorado", "Paramus": "New Jersey", "Minneapolis": "Minnesota", "Country Walk": "Florida", "Tallmadge": "Ohio", "Clearfield": "Utah", "Jamaica": "New York", "Rockford": "Illinois", "Redondo Beach": "California", "Cottage Grove": "Minnesota", "Agoura Hills": "California", "Stanton": "California", "Smithfield": "Rhode Island", "Bixby": "Oklahoma", "Fort Pierce": "Florida", "Adrian": "Michigan", "Bay Point": "California", "Renton": "Washington", "Erie": "Colorado", "Rolling Meadows": "Illinois", "Sparks": "Nevada", "West Albany": "New York", "North Las Vegas": "Nevada", "Atlanta": "Georgia", "Redmond": "Washington", "Passaic": "New Jersey", "Westbrook": "Maine", "Gladstone": "Missouri", "Willimantic": "Connecticut", "Clemmons": "North Carolina", "Woodlawn": "Maryland", "Citrus Park": "Florida", "Compton": "California", "Enid": "Oklahoma", "North Valley Stream": "New York", "Farmington": "Utah", "Alliance": "Ohio", "Twin Falls": "Idaho", "Neenah": "Wisconsin", "Santa Barbara": "California", "Winter Gardens": "California", "Windham": "Connecticut", "Westport": "Connecticut", "Bloomingdale": "Florida", "Romulus": "Michigan", "Donna": "Texas", "Fair Lawn": "New Jersey", "Utica": "New York", "Sierra Vista": "Arizona", "South Plainfield": "New Jersey", "Rogers": "Arkansas", "Seaford": "New York", "Sherrelwood": "Colorado", "Mattoon": "Illinois", "Warren": "Ohio", "Coventry": "Rhode Island", "Billings": "Montana", "Miramar": "Florida", "Carson": "California", "Ukiah": "California", "Ames": "Iowa", "Watsonville": "California", "Richland": "Washington", "Coral Gables": "Florida", "Hot Springs National Park": "Arkansas", "Bainbridge Island": "Washington", "Battle Ground": "Washington", "Lake Worth": "Florida", "Colorado Springs": "Colorado", "Daytona Beach": "Florida", "Forest Hills": "Michigan", "Setauket-East Setauket": "New York", "Salina": "Kansas", "Manteca": "California", "Sandy City": "Utah", "Upper Alton": "Illinois", "Grosse Pointe Woods": "Michigan", "Forest Grove": "Oregon", "Savannah": "Georgia", "Flagami": "Florida", "Kearney": "Nebraska", "Grandview": "Missouri", "Williamsport": "Pennsylvania", "Port Angeles": "Washington", "Coconut Grove": "Florida", "Boone": "North Carolina", "Cheyenne": "Wyoming", "North Bel Air": "Maryland", "South Boston": "Massachusetts", "Logansport": "Indiana", "Flagstaff": "Arizona", "Thousand Oaks": "California", "Roanoke Rapids": "North Carolina", "Takoma Park": "Maryland", "Statesville": "North Carolina", "Palm City": "Florida", "East Orange": "New Jersey", "Waterford": "Michigan", "Maple Grove": "Minnesota", "Metairie": "Louisiana", "Alton": "Illinois", "Commack": "New York", "Rancho Santa Margarita": "California", "East Massapequa": "New York", "Wahiawa": "Hawaii", "Gatesville": "Texas", "Romeoville": "Illinois", "Peoria": "Arizona", "Tumwater": "Washington", "North Fort Myers": "Florida", "Overland": "Missouri", "Monroeville": "Pennsylvania", "Holladay": "Utah", "Palo Alto": "California", "Meads": "Kentucky", "North Chicago": "Illinois", "Marshfield": "Wisconsin", "Clearlake": "California", "Napa": "California", "Hamilton": "Ohio", "Corsicana": "Texas", "North Miami": "Florida", "Franklin Square": "New York", "Rocklin": "California", "Germantown": "Tennessee", "Weirton Heights": "West Virginia", "Irvine": "California", "Oviedo": "Florida", "Summerlin South": "Nevada", "Redlands": "California", "Miamisburg": "Ohio", "Oregon": "Ohio", "Simpsonville": "South Carolina", "Belleville": "Illinois", "Waterville": "Maine", "Zion": "Illinois", "Anderson": "South Carolina", "Oconomowoc": "Wisconsin", "Twentynine Palms": "California", "South Orange": "New Jersey", "Los Angeles": "California", "Clive": "Iowa", "Fairborn": "Ohio", "Linden": "New Jersey", "San Luis Obispo": "California", "Natchez": "Mississippi", "Cutler": "Florida", "Wilkes-Barre": "Pennsylvania", "Cottonwood Heights": "Utah", "Palisades Park": "New Jersey", "Springdale": "Arkansas", "Waltham": "Massachusetts", "East Point": "Georgia", "Yorkville": "Illinois", "Lindenwold": "New Jersey", "El Dorado Hills": "California", "Trenton": "Michigan", "Pomona": "California", "Castlewood": "Colorado", "Springville": "Utah", "North Massapequa": "New York", "Yucca Valley": "California", "Port Saint Lucie": "Florida", "Boyle Heights": "California", "Elmhurst": "Illinois", "Sapulpa": "Oklahoma", "Durango": "Colorado", "Eugene": "Oregon", "Prospect Heights": "Illinois", "Orlando": "Florida", "Lents": "Oregon", "Maricopa": "Arizona", "Saco": "Maine", "Asheboro": "North Carolina", "Alsip": "Illinois", "Medina": "Ohio", "Vancouver": "Washington", "Lake Ronkonkoma": "New York", "Coralville": "Iowa", "Kirkwood": "Missouri", "Joliet": "Illinois", "Austintown": "Ohio", "Chino Hills": "California", "Carrollwood Village": "Florida", "West Pensacola": "Florida", "Coney Island": "New York", "Eagle River": "Alaska", "Costa Mesa": "California", "Back Mountain": "Pennsylvania", "Colton": "California", "Wayne": "New Jersey", "Muskego": "Wisconsin", "Hopkins": "Minnesota", "Greenwood": "South Carolina", "Duluth": "Minnesota", "Amherst Center": "Massachusetts", "Pontiac": "Michigan", "West Saint Paul": "Minnesota", "Union Hill-Novelty Hill": "Washington", "Middleton": "Wisconsin", "Holden": "Massachusetts", "Woonsocket": "Rhode Island", "Houston": "Texas", "Pittsburgh": "Pennsylvania", "Berkeley": "California", "Barstow Heights": "California", "Rialto": "California", "Arlington Heights": "Illinois", "Melville": "New York", "Bellwood": "Illinois", "Cutler Ridge": "Florida", "Pawtucket": "Rhode Island", "Farmingville": "New York", "Willowbrook": "California", "Enterprise": "Nevada", "El Monte": "California", "Glen Ellyn": "Illinois", "Haines City": "Florida", "Wade Hampton": "South Carolina", "College Station": "Texas", "Tuscaloosa": "Alabama", "Cape Coral": "Florida", "DeSoto": "Texas", "North Decatur": "Georgia", "Bridgeview": "Illinois", "Decatur": "Georgia", "Darien": "Illinois", "Gary": "Indiana", "Laramie": "Wyoming", "Santa Paula": "California", "Grayslake": "Illinois", "Enchanted Hills": "New Mexico", "Wilson": "North Carolina", "Miami Lakes": "Florida", "Brookfield": "Wisconsin", "Plum": "Pennsylvania", "Adelanto": "California", "Harvey": "Louisiana", "Bellview": "Florida", "Clearwater": "Florida", "Parma": "Ohio", "Farragut": "Tennessee", "Saint Joseph": "Missouri", "Norton Shores": "Michigan", "Crystal": "Minnesota", "Montebello": "California", "South Saint Paul": "Minnesota", "Ann Arbor": "Michigan", "Plant City": "Florida", "Dana Point": "California", "Cinco Ranch": "Texas", "Centennial": "Colorado", "Monterey": "California", "Lexington": "Kentucky", "Chula Vista": "California", "Bettendorf": "Iowa", "Gaithersburg": "Maryland", "Ogden": "Utah", "Prescott Valley": "Arizona", "Blue Springs": "Missouri", "Brooklyn Park": "Minnesota", "Pacifica": "California", "South Milwaukee": "Wisconsin", "Owensboro": "Kentucky", "Myrtle Grove": "Florida", "West Chicago": "Illinois", "Hobbs": "New Mexico", "Grand Prairie": "Texas", "Wesley Chapel": "Florida", "La Vista": "Nebraska", "Punta Gorda Isles": "Florida", "Westford": "Massachusetts", "Boca Del Mar": "Florida", "Horizon City": "Texas", "Ashland": "Massachusetts", "Ocean Springs": "Mississippi", "Pine Hills": "Florida", "Uniondale": "New York", "Oregon City": "Oregon", "Fillmore": "California", "Wailuku": "Hawaii", "Wadsworth": "Ohio", "Forest Lake": "Minnesota", "San Carlos Park": "Florida", "North Chicopee": "Massachusetts", "Phenix City": "Alabama", "Opelika": "Alabama", "Round Lake": "Illinois", "Bellingham": "Washington", "Eastchester": "New York", "Barnstable": "Massachusetts", "Newton": "Kansas", "Granite City": "Illinois", "Buena Park": "California", "Sayreville Junction": "New Jersey", "Beaver Dam": "Wisconsin", "Yorba Linda": "California", "Rancho San Diego": "California", "Hayesville": "Oregon", "Burlingame": "California", "Inglewood-Finn Hill": "Washington", "Bayshore Gardens": "Florida", "Camden": "New Jersey", "Allison Park": "Pennsylvania", "Port Hueneme": "California", "West Hollywood": "California", "Leavenworth": "Kansas", "Lake Charles": "Louisiana", "Westchase": "Florida", "Glassmanor": "Maryland", "Arden-Arcade": "California", "Azusa": "California", "Fontana": "California", "North Arlington": "New Jersey", "East Moline": "Illinois", "Ithaca": "New York", "Lutz": "Florida", "West Melbourne": "Florida", "East Peoria": "Illinois", "Bourbonnais": "Illinois", "Badger": "Alaska", "North Ogden": "Utah", "Columbine": "Colorado", "Sebastian": "Florida", "East Lake-Orient Park": "Florida", "Morganton": "North Carolina", "Bayside": "California", "Sunrise": "Florida", "Atascadero": "California", "Athens": "Georgia", "Batavia": "Illinois", "Layton": "Utah", "Hueytown": "Alabama", "Benbrook": "Texas", "Mansfield": "Ohio", "Hempstead": "New York", "Menomonie": "Wisconsin", "Reynoldsburg": "Ohio", "Northampton": "Massachusetts", "Asheville": "North Carolina", "Seal Beach": "California", "Dixon": "Illinois", "Butte-Silver Bow (Balance)": "Montana", "Bradley": "Illinois", "Melbourne": "Florida", "Mesa": "Arizona", "Pooler": "Georgia", "Carlisle": "Pennsylvania", "Easley": "South Carolina", "East Los Angeles": "California", "Aventura": "Florida", "Cypress": "California", "Ramona": "California", "Palm Bay": "Florida", "Avondale": "Arizona", "Kingsland": "Georgia", "Sugar Hill": "Georgia", "Galveston": "Texas", "Dyersburg": "Tennessee", "Bethpage": "New York", "Canton": "Georgia", "Altadena": "California", "Randolph": "Massachusetts", "Rutland": "Vermont", "West Sacramento": "California", "Toledo": "Ohio", "Elkhart": "Indiana", "Elk Grove Village": "Illinois", "Malden": "Massachusetts", "Lima": "Ohio", "Shakopee": "Minnesota", "Mayfield Heights": "Ohio", "Willow Grove": "Pennsylvania", "Ironville": "Kentucky", "Arvin": "California", "Bryant": "Arkansas", "Converse": "Texas", "Griffin": "Georgia", "Shreveport": "Louisiana", "Cheshire": "Connecticut", "Sayreville": "New Jersey", "El Reno": "Oklahoma", "Baldwin": "New York", "Maumelle": "Arkansas", "North Tustin": "California", "Grapevine": "Texas", "Wenatchee": "Washington", "Saint Petersburg": "Florida", "Oro Valley": "Arizona", "Hibbing": "Minnesota", "Crystal Lake": "Illinois", "Apex": "North Carolina", "Glenvar Heights": "Florida", "Turlock": "California", "Princeton": "Florida", "Deerfield": "Illinois", "Modesto": "California", "Massapequa": "New York", "Bay Shore": "New York", "Gilroy": "California", "Sun City Center": "Florida", "Freeport": "New York", "Barberton": "Ohio", "Creve Coeur": "Missouri", "Norwalk": "California", "Portland": "Oregon", "Elizabeth City": "North Carolina", "West Puente Valley": "California", "Tonawanda": "New York", "San Jacinto": "California", "Quincy": "Massachusetts", "Kailua": "Hawaii", "Spokane": "Washington", "Oak Forest": "Illinois", "Torrance": "California", "Overland Park": "Kansas", "Security-Widefield": "Colorado", "Starkville": "Mississippi", "Ives Estates": "Florida", "Pensacola": "Florida", "Schofield Barracks": "Hawaii", "Lenoir": "North Carolina", "New Orleans": "Louisiana", "Kennesaw": "Georgia", "Rosedale": "Maryland", "Safety Harbor": "Florida", "Lowell": "Massachusetts", "Yucaipa": "California", "Mitchell": "South Dakota", "Cleburne": "Texas", "Kansas City": "Kansas", "DeKalb": "Illinois", "Tamiami": "Florida", "Chicago Heights": "Illinois", "Granger": "Indiana", "Marysville": "Washington", "Tukwila": "Washington", "Wyckoff": "New Jersey", "Boardman": "Ohio", "Trotwood": "Ohio", "Chatsworth": "California", "Wilmette": "Illinois", "Michigan City": "Indiana", "Hickory": "North Carolina", "Hammond": "Indiana", "Hanover": "Massachusetts", "Coral Terrace": "Florida", "Downers Grove": "Illinois", "North Miami Beach": "Florida", "Winter Park": "Florida", "Garfield Heights": "Ohio", "Poughkeepsie": "New York", "Des Moines": "Iowa", "Fort Smith": "Arkansas", "White Bear Lake": "Minnesota", "Somerville": "Massachusetts", "Santa Monica": "California", "Coral Springs": "Florida", "Faribault": "Minnesota", "Hawthorne": "California", "Sanford": "Florida", "Concord": "Missouri", "Mount Laurel": "New Jersey", "Lake Shore": "Maryland", "Lackawanna": "New York", "Saint Louis Park": "Minnesota", "Northport": "Alabama", "Westchester": "Florida", "Indianapolis": "Indiana", "Gardena": "California", "Country Club Hills": "Illinois", "Pendleton": "Oregon", "Peachtree City": "Georgia", "Landover": "Maryland", "Paso Robles": "California", "Hyattsville": "Maryland", "Port Orange": "Florida", "Bethlehem": "Pennsylvania", "Hermosa Beach": "California", "Indio": "California", "Riverbank": "California", "Sacramento": "California", "Wauwatosa": "Wisconsin", "Conyers": "Georgia", "Merced": "California", "Allendale": "Michigan", "Elmwood Park": "Illinois", "Tarpon Springs": "Florida", "Hialeah": "Florida", "Richfield": "Minnesota", "Morristown": "New Jersey", "Penn Hills": "Pennsylvania", "Winona": "Minnesota", "Woodmere": "New York", "Rutherford": "New Jersey", "Kannapolis": "North Carolina", "Bridgeport": "Connecticut", "Whitehall": "Ohio", "Ruskin": "Florida", "Arnold": "Missouri", "Oakland": "California", "Leawood": "Kansas", "Odenton": "Maryland", "Conroe": "Texas", "Mira Loma": "California", "Braintree": "Massachusetts", "Hurst": "Texas", "Lubbock": "Texas", "York": "Pennsylvania", "Bergenfield": "New Jersey", "Bethel Park": "Pennsylvania", "Normal": "Illinois", "Burnsville": "Minnesota", "Gilbert": "Arizona", "Mount Prospect": "Illinois", "San Clemente": "California", "Taylorsville": "Utah", "Kenner": "Louisiana", "Draper": "Utah", "Elizabethtown": "Kentucky", "Lansing": "Michigan", "Woburn": "Massachusetts", "Weymouth": "Massachusetts", "Bismarck": "North Dakota", "Avon Lake": "Ohio", "South River": "New Jersey", "Berea": "Ohio", "Marina": "California", "Buffalo": "Minnesota", "College Park": "Maryland", "Sylvania": "Ohio", "Milpitas": "California", "Reno": "Nevada", "Seattle": "Washington", "Jasmine Estates": "Florida", "Florida Ridge": "Florida", "Cibolo": "Texas", "Sulphur": "Louisiana", "East Ridge": "Tennessee", "Eagle Mountain": "Utah", "Spanaway": "Washington", "El Centro": "California", "Norco": "California", "Wekiwa Springs": "Florida", "Barrington": "Rhode Island", "Cerritos": "California", "Vineland": "New Jersey", "Mokena": "Illinois", "Belvedere Park": "Georgia", "Niagara Falls": "New York", "Green Valley": "Arizona", "Scotch Plains": "New Jersey", "Woodstock": "Illinois", "Hillcrest Heights": "Maryland", "Lorain": "Ohio", "Dracut": "Massachusetts", "Goodyear": "Arizona", "Wilkinsburg": "Pennsylvania", "Brookline": "Massachusetts", "Albany": "New York", "Madisonville": "Kentucky", "Carney": "Maryland", "Aiken": "South Carolina", "Sedalia": "Missouri", "Bend": "Oregon", "Temecula": "California", "Islip": "New York", "Wichita": "Kansas", "Cooper City": "Florida", "Fort Thomas": "Kentucky", "Stow": "Ohio", "Millville": "New Jersey", "Casper": "Wyoming", "Camarillo": "California", "Loma Linda": "California", "Highland Village": "Texas", "Miami": "Florida", "Avenel": "New Jersey", "Deltona": "Florida", "Margate": "Florida", "Sarasota": "Florida", "Derry": "New Hampshire", "Echo Park": "California", "The Bronx": "New York", "Cortland": "New York", "Elyria": "Ohio", "The Hammocks": "Florida", "Norman": "Oklahoma", "Cumberland": "Maryland", "La Crescenta-Montrose": "California", "Chelsea": "Massachusetts", "Peabody": "Massachusetts", "Mesquite": "Nevada", "Macomb": "Illinois", "Rowland Heights": "California", "Marquette": "Michigan", "Belle Glade": "Florida", "North Potomac": "Maryland", "West Islip": "New York", "Rome": "Georgia", "Alexandria": "Louisiana", "Spanish Fork": "Utah", "Fern Creek": "Kentucky", "Montgomery Village": "Maryland", "San Tan Valley": "Arizona", "Panama City": "Florida", "Cloverly": "Maryland", "Los Altos": "California", "South San Jose Hills": "California", "Fairhope": "Alabama", "Allen": "Texas", "West Mifflin": "Pennsylvania", "Havelock": "North Carolina", "Oxford": "Mississippi", "Provo": "Utah", "Alum Rock": "California", "Frisco": "Texas", "San Gabriel": "California", "Pleasant Grove": "Utah", "South Jordan": "Utah", "Yuba City": "California", "Rye": "New York", "Lee's Summit": "Missouri", "Murrieta": "California", "Round Lake Beach": "Illinois", "Newport Beach": "California", "Rosemount": "Minnesota", "Stoughton": "Massachusetts", "Pueblo West": "Colorado", "Edgewood": "Maryland", "Gautier": "Mississippi", "Brenham": "Texas", "Dumont": "New Jersey", "Fort Worth": "Texas", "Tillmans Corner": "Alabama", "Collinsville": "Illinois", "Port Charlotte": "Florida", "Westmont": "California", "Dover": "Delaware", "Long Branch": "New Jersey", "DeLand": "Florida", "Moline": "Illinois", "Rossville": "Maryland", "West Linn": "Oregon", "Brentwood Estates": "Tennessee", "Minnetonka": "Minnesota", "Hanahan": "South Carolina", "Makakilo City": "Hawaii", "Pikesville": "Maryland", "Dubuque": "Iowa", "Tupelo": "Mississippi", "West Seneca": "New York", "Parkville": "Maryland", "Marlborough": "Massachusetts", "Long Island City": "New York", "Mobile": "Alabama", "Lake Zurich": "Illinois", "Fort Lee": "New Jersey", "Merrillville": "Indiana", "Mason": "Ohio", "Hilo": "Hawaii", "Goshen": "Indiana", "Valdosta": "Georgia", "Kingman": "Arizona", "Dyer": "Indiana", "Cedar City": "Utah", "Tanque Verde": "Arizona", "Covina": "California", "North Canton": "Ohio", "Saint Andrews": "South Carolina", "Washington": "Illinois", "Johnston": "Iowa", "Yonkers": "New York", "Perris": "California", "Apple Valley": "Minnesota", "Southgate": "Michigan", "Hilliard": "Ohio", "Foothill Farms": "California", "Deer Park": "New York", "Elmira": "New York", "Fairhaven": "Massachusetts", "South Old Bridge": "New Jersey", "Ewa Gentry": "Hawaii", "Fort Washington": "Maryland", "Solon": "Ohio", "Williamstown": "New Jersey", "Corcoran": "California", "Sioux Falls": "South Dakota", "Elk River": "Minnesota", "Mauldin": "South Carolina", "Beltsville": "Maryland", "McKinleyville": "California", "Bell": "California", "Derry Village": "New Hampshire", "Pittsfield": "Massachusetts", "Humble": "Texas", "Andover": "Minnesota", "Duarte": "California", "Missoula": "Montana", "Stockbridge": "Georgia", "Port Huron": "Michigan", "Spartanburg": "South Carolina", "Bayonet Point": "Florida", "East Concord": "New Hampshire", "Lealman": "Florida", "Warner Robins": "Georgia", "Doral": "Florida", "Ocoee": "Florida", "Cuyahoga Falls": "Ohio", "Oak Creek": "Wisconsin", "Ballenger Creek": "Maryland", "North Port": "Florida", "Idaho Falls": "Idaho", "Wilmington": "North Carolina", "Nixa": "Missouri", "Inglewood": "California", "Kenmore": "Washington", "San Diego": "California", "Elizabeth": "New Jersey", "Baytown": "Texas", "Granite Bay": "California", "Canyon Lake": "Texas", "Blytheville": "Arkansas", "Oakley": "California", "Boca Raton": "Florida", "Shaker Heights": "Ohio", "Fairfield": "Ohio", "Kalamazoo": "Michigan", "Leisure City": "Florida", "Moscow": "Idaho", "Charleston": "South Carolina", "Kings Park": "New York", "Kearns": "Utah", "Cambridge": "Massachusetts", "Laurinburg": "North Carolina", "East Northport": "New York", "Ken Caryl": "Colorado", "Chapel Hill": "North Carolina", "Hialeah Gardens": "Florida", "Rosamond": "California", "Tucker": "Georgia", "Sun City": "Arizona", "South El Monte": "California", "Amherst": "New York", "Desert Hot Springs": "California", "Ronkonkoma": "New York", "Lyndhurst": "New Jersey", "Siloam Springs": "Arkansas", "Watertown": "Massachusetts", "Middleburg Heights": "Ohio", "Bell Gardens": "California", "Hot Springs": "Arkansas", "West Memphis": "Arkansas", "West Scarborough": "Maine", "Holyoke": "Massachusetts", "Fairwood": "Washington", "Pittsburg": "Kansas", "Seabrook": "Maryland", "Youngstown": "Ohio", "Redan": "Georgia", "Atwater": "California", "Anaheim": "California", "Bostonia": "California", "Palm Harbor": "Florida", "Prescott": "Arizona", "East Providence": "Rhode Island", "Edmond": "Oklahoma", "Ocean Acres": "New Jersey", "East Florence": "Alabama", "Hayward": "California", "Needham": "Massachusetts", "Dearborn": "Michigan", "Ramsey": "Minnesota", "Johns Creek": "Georgia", "Wallingford Center": "Connecticut", "Coram": "New York", "West Coon Rapids": "Minnesota", "Bakersfield": "California", "Winston-Salem": "North Carolina", "Slidell": "Louisiana", "Riverton": "Utah", "Ossining": "New York", "Cookeville": "Tennessee", "Tewksbury": "Massachusetts", "Port Chester": "New York", "Rolla": "Missouri", "Rock Hill": "South Carolina", "Graham": "Washington", "East Brunswick": "New Jersey", "Greenville": "Texas", "Fort Collins": "Colorado", "Suwanee": "Georgia", "New City": "New York", "Duncan": "Oklahoma", "Forest Park": "Georgia", "Daphne": "Alabama", "Mountlake Terrace": "Washington", "Memphis": "Tennessee", "The Villages": "Florida", "Nipomo": "California", "Temple City": "California", "Alhambra": "California", "Northfield": "Minnesota", "Bountiful": "Utah", "Tooele": "Utah", "Santa Fe Springs": "California", "University": "Florida", "Fresno": "Texas", "Buckeye": "Arizona", "Allen Park": "Michigan", "Mount Lebanon": "Pennsylvania", "Brook Park": "Ohio", "North Plainfield": "New Jersey", "Hoover": "Alabama", "Benicia": "California", "Hunt Valley": "Maryland", "Sumter": "South Carolina", "Mandan": "North Dakota", "Biddeford": "Maine", "Cartersville": "Georgia", "Bowie": "Maryland", "Yukon": "Oklahoma", "Anacortes": "Washington", "Morton": "Illinois", "Universal City": "California", "Hercules": "California", "Towson": "Maryland", "Sioux City": "Iowa", "Brunswick": "Georgia", "West Hempstead": "New York", "Auburn Hills": "Michigan", "Aspen Hill": "Maryland", "West Hartford": "Connecticut", "Houma": "Louisiana", "Jeffersonville": "Indiana", "Saratoga": "California", "Delano": "California", "Sunny Isles Beach": "Florida", "Saint Peters": "Missouri", "Oak Ridge": "Tennessee", "Leesburg": "Florida", "Florissant": "Missouri", "Hicksville": "New York", "Federal Way": "Washington", "Joplin": "Missouri", "Weston": "Florida", "Davenport": "Iowa", "Abington": "Massachusetts", "Novato": "California", "Laplace": "Louisiana", "Portsmouth": "New Hampshire", "Eden Prairie": "Minnesota", "Mableton": "Georgia", "Sunrise Manor": "Nevada", "Campbell": "California", "Waukegan": "Illinois", "Poplar Bluff": "Missouri", "Midland": "Texas", "Pascagoula": "Mississippi", "Ottumwa": "Iowa", "Walker": "Michigan", "Danville": "Illinois", "West Carson": "California", "Hereford": "Texas", "Wilton": "Connecticut", "San Bernardino": "California", "Encinitas": "California", "Los Gatos": "California", "Sheridan": "Wyoming", "Burbank": "Illinois", "Riverdale": "Georgia", "Savage": "Minnesota", "Galt": "California", "Marana": "Arizona", "Palos Hills": "Illinois", "Thornton": "Colorado", "Hermitage": "Pennsylvania", "McHenry": "Illinois", "Keene": "New Hampshire", "Estero": "Florida", "Cleveland": "Ohio", "West Lake Sammamish": "Washington", "Royal Oak": "Michigan", "Superior": "Wisconsin", "Lynn Haven": "Florida", "Green Bay": "Wisconsin", "Makakilo": "Hawaii", "Denton": "Texas", "Euclid": "Ohio", "Eagan": "Minnesota", "Northglenn": "Colorado", "Duncanville": "Texas", "Bethany": "Oklahoma", "Oceanside": "New York", "Pocatello": "Idaho", "Cicero": "Illinois", "Keizer": "Oregon", "El Cerrito": "California", "New Lenox": "Illinois", "Bozeman": "Montana", "Beverly": "Massachusetts", "Niles": "Ohio", "Streamwood": "Illinois", "Kirkland": "Washington", "West Allis": "Wisconsin", "Springboro": "Ohio", "North Druid Hills": "Georgia", "Durant": "Oklahoma"}

In [45]:
list_ = set(list(city_to_state_dict.keys()) + list(city_to_state_dict.values()))
list_ = [place.lower() for place in list_]

In [46]:
city_to_state_dict = {k.lower():v.lower() for k,v in city_to_state_dict.items()}

In [47]:
state_list = []
for city in df.CITY.to_list():
    if city.lower() in city_to_state_dict.keys():
        state_list.append(city_to_state_dict[city.lower()])
    else:
        state_list.append(None)

In [48]:
state_list[3] = "pennsylvania"
state_list[4] = "new york"
state_list[5] = "california"
state_list[6] = "pennsylvania"
state_list[16] = "indiana"
state_list[27] = "pennsylvania"

In [49]:
Counter(state_list)

Counter({'pennsylvania': 47986,
         'california': 41554,
         'texas': 23584,
         None: 20726,
         'indiana': 19332,
         'illinois': 18205,
         'georgia': 13124,
         'florida': 7466,
         'michigan': 3984,
         'massachusetts': 2111,
         'new york': 1482,
         'ohio': 1129,
         'louisiana': 900,
         'colorado': 460,
         'kentucky': 396})

In [50]:
df['STATE'] = state_list

In [18]:
state_to_demo_dict = {"colorado" : "west", "pennsylvania" : "east", "texas" : "south", "california" : "west", "indiana" : "central", 
                      "illinois" : "central", "massachusetts" : "east", "ohio" : "central", "louisiana" : "south", "georgia" : "south",
                      "kentucky" : "central", "michigan" : "north", "florida" : "south", "new york" : "east"}

In [19]:
demographics = []
for state in df.STATE:
    demographics.append(state_to_demo_dict[state])

In [20]:
df['DEMO'] = demographics

In [21]:
print("Number of Ads:", df[df['DEMO']=="east"].ADS.sum())
print("Number of Images:", df[df['DEMO']=="east"].IMAGES.sum())
df[df['DEMO']=="east"]

Number of Ads: 145926
Number of Images: 3459


,CITY,ADS,PHONES,IMAGES,STATE,DEMO
1,Scranton,6063,10969,0,pennsylvania,east
2,Harrisburg,6055,11682,0,pennsylvania,east
3,Poconos,2260,4246,0,pennsylvania,east
4,New York,20273,32129,3459,new york,east
6,Penn State,1175,2231,0,pennsylvania,east
9,York,3322,6388,0,pennsylvania,east
11,Philadelphia,66385,153583,0,pennsylvania,east
14,Williamsport,993,2279,0,pennsylvania,east
17,Watertown,1319,2362,0,massachusetts,east
22,Altoona,1313,2539,0,pennsylvania,east


In [22]:
print("Number of Ads:", df[df['DEMO']=="west"].ADS.sum())
print("Number of Images:", df[df['DEMO']=="west"].IMAGES.sum())
df[df['DEMO']=="west"]

Number of Ads: 179599
Number of Images: 4262


,CITY,ADS,PHONES,IMAGES,STATE,DEMO
0,Erie,3939,7420,0,colorado,west
5,San Fransisco,15331,20962,4262,california,west
8,Bakersfield,9380,16949,0,california,west
21,Chico,2405,4542,0,california,west
39,Los Angeles,148544,286177,0,california,west


In [23]:
print("Number of Ads:", df[df['DEMO']=="north"].ADS.sum())
print("Number of Images:", df[df['DEMO']=="north"].IMAGES.sum())
df[df['DEMO']=="north"]

Number of Ads: 7472
Number of Images: 1874


,CITY,ADS,PHONES,IMAGES,STATE,DEMO
33,Detroit,7472,11301,1874,michigan,north


In [24]:
print("Number of Ads:", df[df['DEMO']=="south"].ADS.sum())
print("Number of Images:", df[df['DEMO']=="south"].IMAGES.sum())
df[df['DEMO']=="south"]

Number of Ads: 95087
Number of Images: 17393


,CITY,ADS,PHONES,IMAGES,STATE,DEMO
7,Dallas,21324,33705,4872,texas,south
23,Lafayette,2613,4884,0,louisiana,south
25,Atlanta,26532,41844,6304,georgia,south
30,Houston,22964,34896,6217,texas,south
32,Fresno,1569,2758,0,texas,south
35,Westchester,20085,37114,0,florida,south


In [25]:
print("Number of Ads:", df[df['DEMO']=="central"].ADS.sum())
print("Number of Images:", df[df['DEMO']=="central"].IMAGES.sum())
df[df['DEMO']=="central"]

Number of Ads: 85621
Number of Images: 8239


,CITY,ADS,PHONES,IMAGES,STATE,DEMO
10,Muncie,2282,4152,0,indiana,central
12,South Bend,6747,12656,0,indiana,central
13,Terre Haute,1251,2386,0,indiana,central
15,Chicago,31226,48243,8239,illinois,central
16,Ft Wayne,7285,13487,0,indiana,central
18,Indianapolis,25503,58398,0,indiana,central
19,Kokomo,1631,2978,0,indiana,central
20,Lancaster,4367,8225,0,ohio,central
26,Evansville,2974,5406,0,indiana,central
28,Richmond,622,1211,0,kentucky,central


# Merging the data based on demographics

In [17]:
east = ['Scranton', 'Harrisburg', 'Poconos', 'New York', 'Penn State', 'York', 'Philadelphia', 'Williamsport', 'Watertown', 'Altoona', 'Reading', 'Meadville', 'Pittsburgh', 'Allentown', 'Chambersburg', 'Albany', 'Binghamton']
west = ['Erie', 'San Fransisco', 'Bakersfield', 'Chico', 'Los Angeles']
north = ['Detroit']
south = ['Dallas', 'Lafayette', 'Atlanta', 'Houston', 'Fresno', 'Westchester']
central = ['Muncie', 'South Bend', 'Terre Haute', 'Chicago', 'Ft Wayne', 'Indianapolis', 'Kokomo', 'Lancaster', 'Evansville', 'Richmond', 'Bloomington']

In [18]:
def merge_data_based_on_demographics(demography, data_dir):
    data_df = []
    for city in demography:
        df_ = pd.read_csv(os.path.join(data_dir, city + ".csv")).rename(columns={'body': 'TEXT'})
        df_['CITY'] = [city] * df_.shape[0]
        if 'IMAGES' in df_.columns:
            data_df.append(df_[['TEXT', 'PHONES', 'CITY', 'IMAGES']])
        else:
            data_df.append(df_[['TEXT', 'PHONES', 'CITY']])
    return data_df

In [19]:
east_df = merge_data_based_on_demographics(east, "../data/structured/TEXT")
east_df = pd.concat(east_df)

west_df = merge_data_based_on_demographics(west, "../data/structured/TEXT")
west_df = pd.concat(west_df)

north_df = merge_data_based_on_demographics(north, "../data/structured/TEXT")
north_df = pd.concat(north_df)

south_df = merge_data_based_on_demographics(south, "../data/structured/TEXT")
south_df = pd.concat(south_df)

central_df = merge_data_based_on_demographics(central, "../data/structured/TEXT")
central_df = pd.concat(central_df)

In [ ]:
east_df['DEMO'] = ["east"] * east_df.shape[0]
west_df['DEMO'] = ["west"] * west_df.shape[0]
north_df['DEMO'] = ["north"] * north_df.shape[0]
south_df['DEMO'] = ["south"] * south_df.shape[0]
central_df['DEMO'] = ["central"] * central_df.shape[0]

df = pd.concat([east_df, west_df, north_df, south_df, central_df])

In [21]:
df

,TEXT,PHONES,CITY,IMAGES,DEMO
0,Mixed Goddess looking to pamper and charm you...,"['484-838-3573', '(484)838-3573', '484-8332745...",Scranton,NaN,east
1,Where is the Compassion/Companionship now a d...,['23?? 570-285-4013'],Scranton,NaN,east
2,My name is Mandy. I am available for upscale ...,['(570)351-2749'],Scranton,NaN,east
3,Hey Guys?? I offer a completely safe and non-...,['5703903699'],Scranton,NaN,east
4,barbie 570 814 9924 200 hr flat rate 135 half...,['570 814 9924 200'],Scranton,NaN,east
...,...,...,...,...,...
1728,[SEP] http://posting.bloomington.backpage.co...,"['39002999', '39002999']",Bloomington,NaN,central
1729,[SEP] http://bloomington.backpage.com/Female...,"['3900287139', '287139002999']",Bloomington,NaN,central
1730,[SEP] http://bloomington.backpage.com/Female...,"['3900287539', '287539002999']",Bloomington,NaN,central
1731,[SEP] http://bloomington.backpage.com/Female...,"['3900287939', '287939002999']",Bloomington,NaN,central


In [ ]:
east_df.to_csv("../data/structured/TEXT/east.csv")
west_df.to_csv("../data/structured/TEXT/west.csv")
north_df.to_csv("../data/structured/TEXT/north.csv")
south_df.to_csv("../data/structured/TEXT/south.csv")
central_df.to_csv("../data/structured/TEXT/central.csv")

# Creating communities based on phone numbers

In [22]:
# Using recursion to merge all sublist having common phones and finding communities.
def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current

In [23]:
def assign_vendor_id_to_ads(data_df):
    all_phones = [eval(phones) for phones in data_df['PHONES'].to_list()]
    # All phone numbers should be atleast 7 digits long, any US based phone number is atleast 7 digits long 
    all_phones = [[number for number in numbers if len(number) >= 7] for numbers in all_phones]
    all_phones = filter(None, all_phones)

    graph = to_graph(all_phones)
    communities = connected_components(graph)
    phone_communities = [list(i) for i in communities]
    return phone_communities

In [26]:
all_communities = assign_vendor_id_to_ads(df)

In [27]:
def assign_phone_to_vendor_ids(phone_communities, df):
    # Converting communities of community IDs
    community_ids = {}

    for index, community in enumerate(phone_communities):
        for number in community:
            community_ids[number] = index+1
    
    vendor_id_list = []
    for index, row in df.iterrows():
        vendor = []
        for number in eval(row['PHONES']):
            if number in community_ids.keys():
                vendor.append(community_ids[number])
            else:
                vendor.append(0)
                
        vendor = Counter(vendor).most_common(1)[0][0]
        vendor_id_list.append(vendor)
    
    df['VENDOR'] = vendor_id_list
    return df

In [28]:
df = assign_phone_to_vendor_ids(all_communities, df)

# Visualizing data

In [32]:
vendor_dict = {k:v for k,v in dict(Counter(df['VENDOR'])).items() if v>=5}

In [33]:
df = df.loc[df['VENDOR'].isin(list(vendor_dict.keys()))]

In [34]:
print(df.shape[0], len(set(df.VENDOR.to_list())))

202439 12344


# Saving data into east, west, north, south, and central-american demographics

In [35]:
df.columns

Index(['TEXT', 'PHONES', 'CITY', 'IMAGES', 'DEMO', 'VENDOR'], dtype='object')

In [36]:
east_df = df[df['DEMO']=='east']
west_df = df[df['DEMO']=='west']
north_df = df[df['DEMO']=='north']
south_df = df[df['DEMO']=='south']
central_df = df[df['DEMO']=='central']

In [37]:
east_df.shape, west_df.shape, north_df.shape, south_df.shape, central_df.shape

((62254, 6), (50593, 6), (3984, 6), (45074, 6), (40534, 6))

In [38]:
east_df.to_csv("../data/structured/TEXT/east.csv")
west_df.to_csv("../data/structured/TEXT/west.csv")
north_df.to_csv("../data/structured/TEXT/north.csv")
south_df.to_csv("../data/structured/TEXT/south.csv")
central_df.to_csv("../data/structured/TEXT/central.csv")

# Plotting the US plot for ads density

In [58]:
east_df = pd.read_csv("../data/structured/TEXT/east.csv", low_memory=False)
west_df = pd.read_csv("../data/structured/TEXT/west.csv", low_memory=False)
north_df = pd.read_csv("../data/structured/TEXT/north.csv", low_memory=False)
south_df = pd.read_csv("../data/structured/TEXT/south.csv", low_memory=False)
central_df = pd.read_csv("../data/structured/TEXT/central.csv", low_memory=False)

df = pd.concat([east_df, west_df, north_df, south_df, central_df])

In [52]:
df.columns

Index(['Unnamed: 0', 'TEXT', 'PHONES', 'CITY', 'IMAGES', 'DEMO', 'VENDOR',
       'STATE'],
      dtype='object')

In [53]:
import pandas as pd

df_ = pd.read_csv(r'../data/state_market_tracker.tsv000', sep='\t', usecols=['state','state_code'])
df_['state'] = df_['state'].apply(lambda x: x.lower())

In [54]:
df_ = df_.set_index('state')
df_dict = df_.to_dict()

In [57]:
df

,state,state_code,ads
0,pennsylvania,NaN,NaN


In [55]:
state_list, ads_list, state_code_list = ([] for i in range(3))
all_states = df.STATE.unique()

for state in all_states:
    temp_df = df[df['STATE']==state]
    state_list.append(state)
    ads_list.append(temp_df.ADS.sum())
    state_code_list.append(df_dict['state_code'][state])

AttributeError: 'DataFrame' object has no attribute 'ADS'

In [56]:
df = pd.DataFrame(columns=['state','state_code','ads'])
df['state'] = state_list
df['state_code'] = state_code_list
df['ads'] = ads_list

ValueError: Length of values (0) does not match length of index (1)

In [31]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [34]:
import plotly.express as px

fig = px.choropleth(df,
                    locations='state_code', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='ads',
                    color_continuous_scale="Reds", 
                    )
fig.show()